In [137]:
'''
IF RECOMPILING: DELETE author_tokens.txt
''' 

'\nIF RECOMPILING: DELETE author_tokens.txt\n'

In [118]:
import glob
import pandas as pd
import nltk
from nltk import word_tokenize
import string
import json
import re
import os

In [119]:
from nltk.tag import StanfordNERTagger

#Check that paths are set properly for local host
jar = './stanford-ner.jar'
model = './english.all.3class.distsim.crf.ser.gz'

ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

In [120]:
#Get file names for all texts
txt_files = glob.glob("*.txt")

In [121]:
'''
Function "load" takes a .txt file as the parameter. It reads the .txt file, removes new line
indicators (\n), and returns the file as a string.
'''
def load(text):
    with open(text, encoding="ISO-8859-1") as f:
        my_text = f.read()
        lines = my_text.replace('\n',' ')
        return lines

In [122]:
#Create list of unique authors (142 total)
author_names = []
for i in range(len(txt_files)):
    string = txt_files[i]
    author_names.append(string[:string.index("_")])

In [123]:
#Load all txt files (3,036 total) in list 
work_loaded = []
for work in txt_files:
    work_loaded.append(load(work))

In [124]:
#Replace digits with '0'
digits = []
for word in work_loaded:
    temp_string = "".join(word)
    replaced_string = re.sub('\d', '0', temp_string)
    digits.append(replaced_string)

In [125]:
#Tokenize text
work_tokenized = []
for word in digits:
    work_tokenized.append(word_tokenize(word))

In [127]:
#Replace tokens tagged as PERSON, ORGANIZATION, or LOCATION with '#'
final_tokens = []
for i in range((len(work_tokenized))):
            tag_tokens = ner_tagger.tag(work_tokenized[i])
            tag_df = pd.DataFrame(tag_tokens, columns=['token', 'label'])
            tag_df.loc[tag_df.label == "PERSON", 'token'] = "#"
            tag_df.loc[tag_df.label == "ORGANIZATION", 'token'] = "#"
            tag_df.loc[tag_df.label == "LOCATION", 'token'] = "#"
            new_tokens = list(tag_df['token'])
            final_tokens.append(new_tokens)

In [131]:
#Create dictionary for authors with their corresponding tokens
works = {'author': author_names,
         'work': final_tokens}
author_works_df = pd.DataFrame.from_dict(works)
author_works_dict = {x: y["work"].tolist() for x,y in author_works_df.groupby("author")}

In [138]:
#Write author/tokens dictionary to .txt file
with open('authors_tokens.txt', 'w') as file:
     file.write(json.dumps(author_works_dict))

In [ ]:
'''
for key, value in author_works_dict.items():
    temp_string = str(value)
    replaced_string = re.sub('\d', '0', temp_string)
    author_works_dict[key]= replaced_string
'''